In [ ]:
!pip install datasets deepeval

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 11.5 MB/s

In [ ]:
import os
import argparse

import numpy as np
from datasets import load_dataset
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams
from dotenv import load_dotenv
from openai import OpenAI
from scipy.stats import ttest_rel, ttest_1samp

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

client = OpenAI()

style_matching_metric_geval = GEval(
    name="Style Matching",
    criteria="Определи, насколько последняя фраза (actual output) делает анекдот смешным, логичным и остроумным:",

    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

logical_completion_metric_geval = GEval(
    name="Logical Completion",
    criteria="Оцени, насколько анекдот структурирован, есть ли хороший сетап и панчлайн",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

postironical_metric_geval = GEval(
    name="Postironical Completion",
    criteria="Оцени, насколько анекдот постироничен.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)


system_message1 = {"role": "system",
                   "content": "Ты чат бот, который генерирует анекдоты. "
                              "Продолжи диалог одной репликой так, чтоб получился анекдот. "
                              "Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? "
                              "Expected output: - Мои!"}
system_message2 = {"role": "system",
                   "content": "Ты чат бот, который генерирует диалоги. Продолжи диалог"}


def prepare_openai_jokes_dataset(args):
    ds = load_dataset("inkoziev/jokes_dialogues")
    ds_df = ds['train'].to_pandas()

    # print(ds_df)
    unique_chat_ids = ds_df['src_hash'].unique()
    # unique_chat_ids = set(ds['train']['src_hash'])
    sampled_chat_ids = np.random.choice(list(unique_chat_ids), args.num_samples, replace=False)

    openai_chat_examples = []
    for chat_id in sampled_chat_ids:
        reply_nums = ds_df[ds_df['src_hash'] == chat_id]['reply_num'].values
        reply_nums = np.sort(reply_nums)
        messages = [system_message1]
        if reply_nums[0] != 1:
            continue
        message1 = ds_df[(ds_df['src_hash'] == chat_id) & (ds_df['reply_num'] == 1)]['context'].values[0]
        messages.append({"role": "assistant" if 0 % 2 == reply_nums[-1] % 2 else "user",
                         "content": message1})

        for reply_num in reply_nums:
            message = ds_df[(ds_df['src_hash'] == chat_id) & (ds_df['reply_num'] == reply_num)]['utterance'].values[0]
            message_metadata = {"role": "assistant" if reply_num % 2 == reply_nums[-1] % 2 else "user",
                                "content": message}
            if message_metadata['role'] == 'assistant':
                if reply_num == reply_nums[-1]:
                    message_metadata['weight'] = 1
                else:
                    message_metadata['weight'] = 0
            messages.append(message_metadata)

        openai_chat_example = {"messages": messages}
        openai_chat_examples.append(openai_chat_example)
    return openai_chat_examples


def get_model_predictions(gt_examples, system_message):
    predicted_examples = []
    for i_example in range(len(gt_examples)):
        completion = client.chat.completions.create(
            model="ft:gpt-4o-mini-2024-07-18:personal::AgMH5gFH",
            messages=[system_message] + gt_examples[i_example]['messages'][1:-1],
        )
        # predicted_examples.append(completion.choices[0].message.content)
        predicted_example = [system_message] + gt_examples[i_example]['messages'][1:-1]
        predicted_example.append({"role": "assistant", "content": completion.choices[0].message.content})
        predicted_example = {"messages": predicted_example}
        predicted_examples.append(predicted_example)

    return predicted_examples


def evaluate_metrics(gt_examples, predicted_examples1, predicted_examples2):
    scores1 = []
    scores2 = []

    logical_scores1 = []
    logical_scores2 = []

    postironical_score1 = []
    postironical_score2 = []

    for i_example in range(len(gt_examples)):
        test_case1 = LLMTestCase(
            input="\n".join([msg['content'] for msg in predicted_examples1[i_example]['messages'][1:-1]]),
            actual_output=predicted_examples1[i_example]['messages'][-1]['content'],
            expected_output=gt_examples[i_example]['messages'][-1]['content'],
        )
        test_case2 = LLMTestCase(
            input="\n".join([msg['content'] for msg in predicted_examples2[i_example]['messages'][1:-1]]),
            actual_output=predicted_examples2[i_example]['messages'][-1]['content'],
            expected_output=gt_examples[i_example]['messages'][-1]['content'],
        )

        style_matching_metric_geval.measure(test_case1)
        predicted_examples1[i_example]['fun_score'] = style_matching_metric_geval.score
        scores1.append(style_matching_metric_geval.score)

        style_matching_metric_geval.measure(test_case2)
        predicted_examples2[i_example]['fun_score'] = style_matching_metric_geval.score
        scores2.append(style_matching_metric_geval.score)

        logical_completion_metric_geval.measure(test_case1)
        logical_scores1.append(logical_completion_metric_geval.score)

        logical_completion_metric_geval.measure(test_case2)
        logical_scores2.append(logical_completion_metric_geval.score)

        postironical_metric_geval.measure(test_case1)
        postironical_score1.append(postironical_metric_geval.score)

        postironical_metric_geval.measure(test_case2)
        postironical_score2.append(postironical_metric_geval.score)

    return gt_examples, predicted_examples1, predicted_examples2, scores1, scores2, logical_scores1, logical_scores2, postironical_score1, postironical_score2


def main(args):
    gt_examples = prepare_openai_jokes_dataset(args)
    print("============== GT EXAMPLES ===============")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print("-----------")
    predicted_examples1 = get_model_predictions(gt_examples, system_message1)
    predicted_examples2 = get_model_predictions(gt_examples, system_message2)
    print("=============== GT and Predicted EXAMPLES ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")
    gt_examples, predicted_examples1, predicted_examples2, scores1, scores2, logical_scores1, logical_scores2, postironical_score1, postironical_score2 = evaluate_metrics(gt_examples,
                                                                                               predicted_examples1,
                                                                                               predicted_examples2)

    print("=============== GT and Predicted EXAMPLES with METRICS ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")

    print("=============== Scores ======================")
    ttest_res1 = ttest_1samp(scores1, 0.5)
    print("SCORE1:", np.mean(scores1), scores1)
    print("TTEST1:", ttest_res1)
    ttest_res2 = ttest_1samp(scores2, 0.5)
    print("SCORE2:", np.mean(scores2), scores2)
    print("TTEST2:", ttest_res2)

    rel_ttest_res = ttest_rel(scores1, scores2)

    print("=============== Rel Scores ======================")
    print("TTEST:", rel_ttest_res)

    print("=============== Logical Scores ======================")
    logical_ttest_res1 = ttest_1samp(logical_scores1, 0.5)
    print("LOGICAL_SCORE1:", np.mean(logical_scores1), logical_scores1)
    print("LOGICAL_TTEST1:", logical_ttest_res1)

    logical_ttest_res2 = ttest_1samp(logical_scores2, 0.5)
    print("LOGICAL_SCORE2:", np.mean(logical_scores2), logical_scores2)
    print("LOGICAL_TTEST2:", logical_ttest_res2)

    logical_rel_ttest_res = ttest_rel(logical_scores1, logical_scores2)
    print("LOGICAL_REL_TTEST:", logical_rel_ttest_res)

    print("=============== Postironical Scores ======================")
    postironical_ttest_res1 = ttest_1samp(postironical_score1, 0.5)
    print("POSTIRONICAL_SCORE1:", np.mean(postironical_score1), postironical_score1)
    print("POSTIRONICAL_TTEST1:", logical_ttest_res1)

    postironical_ttest_res2 = ttest_1samp(postironical_score2, 0.5)
    print("POSTIRONICAL_SCORE2:", np.mean(postironical_score2), postironical_score2)
    print("POSTIRONICAL_TTEST2:", postironical_ttest_res2)

    postironical_rel_ttest_res = ttest_rel(postironical_score1, postironical_score2)
    print("POSTIRONICAL_REL_TTEST:", postironical_rel_ttest_res)



if __name__ == '__main__':
    class Args:
        num_samples = 20
    args = Args()
    main(args)


============== GT EXAMPLES ===============
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- От вас сексом так и фонит.'}, {'role': 'assistant', 'content': 'Я сексофонист.', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Мойша, скажите, вы с Басей счастливы?'}, {'role': 'assistant', 'content': 'А куда деваться?..', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

=============== GT and Predicted EXAMPLES ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- От вас сексом так и фонит.'}, {'role': 'assistant', 'content': 'Я сексофонист.', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- От вас сексом так и фонит.'}, {'role': 'assistant', 'content': 'Извините, это мои внутренние демоны.'}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует диалоги. Продолжи диалог'}, {'role': 'user', 'conten

Output()

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

=============== GT and Predicted EXAMPLES with METRICS ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- От вас сексом так и фонит.'}, {'role': 'assistant', 'content': 'Я сексофонист.', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- От вас сексом так и фонит.'}, {'role': 'assistant', 'content': 'Извините, это мои внутренние демоны.'}], 'fun_score': 0.4248469143376238}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует диалог

До файнтюнинга:

SCORE1 (0.546) немного выше порогового значения 0.5, но всё равно близко к случайному уровню, что подтверждается p-value = 0.166 (мы не можем отвергнуть нулевую гипотезу о том, что средние совпадают). В то время как SCORE2 (0.246) значительно ниже 0.5, что указывает на менее смешные и логичные ответы (p-value = 1.94e-07). Сравнение этих результатов показывает статистически значимое различие (p-value = 2.39e-05), где модель с system_message1 генерирует значительно лучшие ответы, пусть и не самые смешные.

LOGICAL_SCORE1 (0.403) указывает на умеренно логичные, но ещё далёкие от идеала завершения, с p-value = 0.046. LOGICAL_SCORE2 (0.248) значительно ниже 0.5, что указывает на менее логичные завершения (p-value = 3.2e-07). Сравнение этих результатов (p-value = 0.0037) показывает, что модель с system_message1 генерирует более логичные завершения.

После файнтюнинга:

SCORE1 (0.305) значительно ниже порогового значения 0.5, что указывает на близкие к случайным ответы. Это подтверждается крайне низким p-value = 2.19e-09, отвергающим нулевую гипотезу о равенстве средних. В то время как SCORE2 (0.385) также ниже 0.5, но значительно выше SCORE1, что указывает на более качественные ответы (p-value = 0.00048). Сравнение этих результатов показывает статистически значимое различие (p-value = 0.014), где модель с system_message2 генерирует более качественные завершения.

LOGICAL_SCORE1 (0.316) демонстрирует низкий уровень логичности завершений, с p-value = 9.88e-08, что свидетельствует о статистически значимом отличии от случайного уровня. LOGICAL_SCORE2 (0.389) ближе к пороговому значению 0.5, что указывает на несколько более логичные ответы, с p-value = 0.00177. Сравнение этих результатов (p-value = 0.022) подтверждает, что модель с system_message2 генерирует более логичные завершения.

POSTIRONICAL_SCORE1 (0.436) показывает умеренно качественные завершения, находящиеся ближе к приемлемому уровню, но все ещё далёкие от идеала. Значение p-value = 9.88e-08 подтверждает статистическую значимость результата. POSTIRONICAL_SCORE2 (0.504) приближается к пороговому значению 0.5, демонстрируя ещё более качественные завершения. Однако сравнение между POSTIRONICAL_SCORE1 и POSTIRONICAL_SCORE2 (p-value = 0.091) не даёт основания утверждать, что различие статистически значимо.

Эти результаты показывают, что модель с system_message2 лучше справляется с задачами логики и качества завершений, но не демонстрирует явного преимущества в постиронической метрике. Дообученная модель работает лучше в качестве "продолжателя диалога", чем "создатель анекдотов"

Дообученная модель в целом демонстрирует ухудшение качества по метрикам g-eval (за исключением постироничности, данная метрика высока:) ) В первую очередь такой результат связан с не самыми лучшими обучающими данными. Анекдоты в датасете разбиты на части в формате диалога, получение лучших результатов может быть достигнуто путем более тщательной обработки исходного датасета.